<a href="https://colab.research.google.com/github/zhirongwang94/AI-Colorization/blob/master/Colorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
! echo "# AI-Colorization" >> README.md


! git push -u origin master

fatal: could not read Username for 'https://github.com': No such device or address
